In [1]:
import tensorflow 

import pandas as pd
import numpy as np
import os
import keras
import random
import cv2
import math
import seaborn as sns

from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Convolution2D,BatchNormalization
from tensorflow.keras.layers import Flatten,MaxPooling2D,Dropout

from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications.densenet import preprocess_input

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,img_to_array

from tensorflow.keras.models import Model

from tensorflow.keras.optimizers import Adam

from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

import warnings
warnings.filterwarnings("ignore")

In [2]:
print("Tensorflow-version:", tensorflow.__version__)

Tensorflow-version: 2.9.1


In [3]:
model_d=DenseNet121(weights='imagenet',include_top=False, input_shape=(128, 128, 3)) 

x=model_d.output

x= GlobalAveragePooling2D()(x)
x= BatchNormalization()(x)
x= Dropout(0.5)(x)
x= Dense(1024,activation='relu')(x) 
x= Dense(512,activation='relu')(x) 
x= BatchNormalization()(x)
x= Dropout(0.5)(x)

preds=Dense(5,activation='softmax')(x)

In [4]:
model=Model(inputs=model_d.input,outputs=preds)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 134, 134, 3)  0          ['input_1[0][0]']                
                                                                                                  
 conv1/conv (Conv2D)            (None, 64, 64, 64)   9408        ['zero_padding2d[0][0]']         
                                                                                                  
 conv1/bn (BatchNormalization)  (None, 64, 64, 64)   256         ['conv1/conv[0][0]']         

In [6]:
for layer in model.layers[:-8]:
    layer.trainable=False

In [7]:
for layer in model.layers[-8:]:
    layer.trainable=True

In [8]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 134, 134, 3)  0          ['input_1[0][0]']                
                                                                                                  
 conv1/conv (Conv2D)            (None, 64, 64, 64)   9408        ['zero_padding2d[0][0]']         
                                                                                                  
 conv1/bn (BatchNormalization)  (None, 64, 64, 64)   256         ['conv1/conv[0][0]']         

In [9]:
data=[]
labels=[]
random.seed(42)
imagePaths = sorted(list(os.listdir("C:\\Users\\Pujitha Thejaswi\\dataset")))
random.shuffle(imagePaths)
print(imagePaths)

for img in imagePaths:
    path=sorted(list(os.listdir("C:\\Users\\Pujitha Thejaswi\\dataset/"+img)))
    for i in path:
        image = cv2.imread("C:\\Users\\Pujitha Thejaswi\\dataset/"+img+'/'+i)
        image = cv2.resize(image, (128,128))
        image = img_to_array(image)
        data.append(image)
        l = label = img
        labels.append(l)

['Proliferate', 'Moderate', 'Normal', 'Severe', 'Mild']


In [10]:
data = np.array(data, dtype="float32") / 255.0
labels = np.array(labels)
mlb = LabelBinarizer()
labels = mlb.fit_transform(labels)
print(labels[0])

[0 0 0 1 0]


In [11]:
(xtrain,xtest,ytrain,ytest)=train_test_split(data,labels,test_size=0.2,random_state=42)
print(xtrain.shape, xtest.shape, ytest.shape, ytrain.shape)
len(xtrain)

(2929, 128, 128, 3) (733, 128, 128, 3) (733, 5) (2929, 5)


2929

In [12]:
anne = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=5, verbose=1, min_lr=1e-3)
checkpoint = ModelCheckpoint('model.h5', verbose=1, save_best_only=True)

In [16]:
datagen = ImageDataGenerator(rotation_range=45,width_shift_range=0.2,height_shift_range=0.2,zoom_range = 0.2, horizontal_flip=True, shear_range=0.2,fill_mode='nearest')

In [17]:
datagen.fit(xtrain)

In [18]:
history = model.fit_generator(datagen.flow(xtrain, ytrain, batch_size=32),
               steps_per_epoch=xtrain.shape[0] //32,
               epochs=50,
               verbose=2,
               callbacks=[anne, checkpoint],
               validation_data=(xtrain, ytrain))

Epoch 1/50

Epoch 1: val_loss improved from 3.36212 to 0.84751, saving model to model.h5
91/91 - 160s - loss: 0.9266 - accuracy: 0.6735 - val_loss: 0.8475 - val_accuracy: 0.6862 - lr: 0.0010 - 160s/epoch - 2s/step
Epoch 2/50

Epoch 2: val_loss improved from 0.84751 to 0.73700, saving model to model.h5
91/91 - 167s - loss: 0.7891 - accuracy: 0.7069 - val_loss: 0.7370 - val_accuracy: 0.7241 - lr: 0.0010 - 167s/epoch - 2s/step
Epoch 3/50

Epoch 3: val_loss did not improve from 0.73700
91/91 - 164s - loss: 0.7567 - accuracy: 0.7111 - val_loss: 0.8318 - val_accuracy: 0.7105 - lr: 0.0010 - 164s/epoch - 2s/step
Epoch 4/50

Epoch 4: val_loss improved from 0.73700 to 0.68197, saving model to model.h5
91/91 - 165s - loss: 0.7615 - accuracy: 0.7090 - val_loss: 0.6820 - val_accuracy: 0.7375 - lr: 0.0010 - 165s/epoch - 2s/step
Epoch 5/50

Epoch 5: val_loss did not improve from 0.68197
91/91 - 167s - loss: 0.7260 - accuracy: 0.7197 - val_loss: 0.6981 - val_accuracy: 0.7313 - lr: 0.0010 - 167s/epoch 

In [19]:
ypred = model.predict(xtest)

total = 0
accurate = 0
accurateindex = []
wrongindex = []

for i in range(len(ypred)):
    if np.argmax(ypred[i]) == np.argmax(ytest[i]):
        accurate += 1
        accurateindex.append(i)
    else:
        wrongindex.append(i)
        
    total += 1
    
print('Total-test-data;', total, '\taccurately-predicted-data:', accurate, '\t wrongly-predicted-data: ', total - accurate)
print('Accuracy:', round(accurate/total*100, 3), '%')

23/23 [==============================] - 88s 3s/step
Total-test-data; 733 	accurately-predicted-data: 532 	 wrongly-predicted-data:  201
Accuracy: 72.578 %


In [ ]:
Ypred = model.predict(xtest)

Ypred = np.argmax(Ypred, axis=1)
Ytrue = np.argmax(ytest, axis=1)

cm = confusion_matrix(Ytrue, Ypred)
plt.figure(figsize=(12, 12))
ax = sns.heatmap(cm, cmap="rocket_r", fmt=".01f",annot_kws={'size':16}, annot=True, square=True, xticklabels=label, yticklabels=label)
ax.set_ylabel('Actual', fontsize=20)
ax.set_xlabel('Predicted', fontsize=20)

In [ ]:
loss, accuracy = model.evaluate(xtest, ytest, verbose=1)
loss_v, accuracy_v = model.evaluate(xtrain, ytrain, verbose=1)
print("Validation: accuracy = %f  ;  loss_v = %f" % (accuracy_v, loss_v))
print("Test: accuracy = %f  ;  loss = %f" % (accuracy, loss))
model.save("model.h5")
